# Détection de Poubelles Pleines/Vides avec YOLOv9

Ce notebook implémente un modèle de détection d'objets pour prédire si une poubelle est pleine ou vide.

## 3. Import des bibliothèques

In [29]:
import os
import time
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import pandas as pd

## 4. Vérification de la structure du dataset

In [30]:
# Trouver le chemin du dataset téléchargé
dataset_path = "dataset_detection_poubelle"
data_yaml = os.path.join(dataset_path, "data.yaml")


print(f"Dataset téléchargé dans: {dataset_path}")
# Vérifier le fichier data.yaml
data_yaml = os.path.join(dataset_path, "data.yaml")
print(f"\nFichier de configuration: {data_yaml}")

# Afficher le contenu du fichier data.yaml
if os.path.exists(data_yaml):
    with open(data_yaml, 'r') as f:
        print(f.read())

Dataset téléchargé dans: dataset_detection_poubelle

Fichier de configuration: dataset_detection_poubelle\data.yaml
train: train/images
val: valid/images
test: test/images

nc: 2
names: ['poubelle_pleine', 'poubelle_vide']

roboflow:
  workspace: poubelledetection
  project: detection_poubelle-6b1qv
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/poubelledetection/detection_poubelle-6b1qv/dataset/1


In [31]:
splits = ["train", "valid", "test"]
extensions = (".jpg", ".jpeg", ".png")

print("📊 Répartition des données :\n")

for split in splits:
    img_dir = os.path.join(dataset_path, split, "images")
    if os.path.isdir(img_dir):
        num_images = len([f for f in os.listdir(img_dir) if f.lower().endswith(extensions)])
        print(f"{split} : {num_images} images")
    else:
        print(f"{split} : dossier non trouvé")

📊 Répartition des données :

train : 226 images
valid : 32 images
test : 17 images


## 5. Entraînement du modèle YOLOv9

In [32]:
# Charger modèle léger pour petit dataset
model = YOLO("yolov8s.pt")   # ou "yolov9s.pt"

start_time = time.time()
results = model.train(
    data=data_yaml,
    epochs=50,        
    imgsz=640,
    batch=8,          
    patience=50,      
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    shear=2.0,
    flipud=0.1,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.2,
    name="poubelle_pleine_vide",
    save=True,
    plots=True
)
end_time = time.time()

print(f"Durée totale : {(end_time - start_time)/60:.1f} min")


New https://pypi.org/project/ultralytics/8.3.230 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.229  Python-3.11.9 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti with Max-Q Design, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_detection_poubelle\data.yaml, degrees=10, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, nam

In [ ]:
training_time = end_time - start_time
training_hours = int(training_time // 3600)
training_minutes = int((training_time % 3600) // 60)
training_seconds = int(training_time % 60)

print(f"Temps total d'entraînement: {training_hours}h {training_minutes}m {training_seconds}s")


Fin de l'entraînement: 2025-11-22 02:16:15
Temps total d'entraînement: 0h 27m 58s
Temps en secondes: 1678.45s


## 6. Courbes d'apprentissage (Loss et Métriques)

In [ ]:
# Charger les résultats d'entraînement depuis le fichier CSV
results_csv = 'runs/detect/poubelle_pleine_vide7/results.csv'

# Lire les résultats
df_results = pd.read_csv(results_csv)
df_results.columns = df_results.columns.str.strip()  # Nettoyer les noms de colonnes

# Créer une figure avec plusieurs sous-graphiques
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Courbes d\'apprentissage - YOLOv9 Détection de Poubelles', fontsize=16, fontweight='bold')

# 1. Loss (Train & Val)
ax1 = axes[0, 0]
if 'train/box_loss' in df_results.columns:
    ax1.plot(df_results['epoch'], df_results['train/box_loss'], label='Train Box Loss', linewidth=2)
if 'val/box_loss' in df_results.columns:
    ax1.plot(df_results['epoch'], df_results['val/box_loss'], label='Val Box Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Box Loss (Train vs Validation)', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Precision & Recall
ax2 = axes[0, 1]
if 'metrics/precision(B)' in df_results.columns:
    ax2.plot(df_results['epoch'], df_results['metrics/precision(B)'], label='Precision', linewidth=2, color='green')
if 'metrics/recall(B)' in df_results.columns:
    ax2.plot(df_results['epoch'], df_results['metrics/recall(B)'], label='Recall', linewidth=2, color='orange')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Score', fontsize=12)
ax2.set_title('Precision & Recall', fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim([0, 1])

# 3. mAP50 & mAP50-95
ax3 = axes[1, 0]
if 'metrics/mAP50(B)' in df_results.columns:
    ax3.plot(df_results['epoch'], df_results['metrics/mAP50(B)'], label='mAP@50', linewidth=2, color='blue')
if 'metrics/mAP50-95(B)' in df_results.columns:
    ax3.plot(df_results['epoch'], df_results['metrics/mAP50-95(B)'], label='mAP@50-95', linewidth=2, color='red')
ax3.set_xlabel('Epoch', fontsize=12)
ax3.set_ylabel('mAP', fontsize=12)
ax3.set_title('Mean Average Precision', fontsize=13, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_ylim([0, 1])

# 4. All Losses Combined
ax4 = axes[1, 1]
if 'train/box_loss' in df_results.columns:
    ax4.plot(df_results['epoch'], df_results['train/box_loss'], label='Box Loss', linewidth=2)
if 'train/cls_loss' in df_results.columns:
    ax4.plot(df_results['epoch'], df_results['train/cls_loss'], label='Class Loss', linewidth=2)
if 'train/dfl_loss' in df_results.columns:
    ax4.plot(df_results['epoch'], df_results['train/dfl_loss'], label='DFL Loss', linewidth=2)
ax4.set_xlabel('Epoch', fontsize=12)
ax4.set_ylabel('Loss', fontsize=12)
ax4.set_title('Toutes les Loss d\'entraînement', fontsize=13, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

<Figure size 1600x1200 with 4 Axes>

In [42]:
# Afficher les statistiques finales
print("\n" + "="*60)
print("STATISTIQUES FINALES D'ENTRAÎNEMENT")
print("="*60)
last_epoch = df_results.iloc[-1]
print(f"\nÉpoque finale: {int(last_epoch['epoch'])}")
if 'metrics/mAP50(B)' in df_results.columns:
    print(f"mAP@50: {last_epoch['metrics/mAP50(B)']:.4f}")
if 'metrics/mAP50-95(B)' in df_results.columns:
    print(f"mAP@50-95: {last_epoch['metrics/mAP50-95(B)']:.4f}")
if 'metrics/precision(B)' in df_results.columns:
    print(f"Precision: {last_epoch['metrics/precision(B)']:.4f}")
if 'metrics/recall(B)' in df_results.columns:
    print(f"Recall: {last_epoch['metrics/recall(B)']:.4f}")


STATISTIQUES FINALES D'ENTRAÎNEMENT

Époque finale: 50
mAP@50: 0.9119
mAP@50-95: 0.5706
Precision: 0.8552
Recall: 0.8947


## 7. Évaluation du modèle sur l'ensemble de validation

In [37]:
# Charger le meilleur modèle
best_model = YOLO('runs/detect/poubelle_pleine_vide7/weights/best.pt')

# Évaluer le modèle sur l'ensemble de validation
print("Évaluation du modèle sur l'ensemble de validation...")
metrics = best_model.val()

# Afficher les métriques détaillées
print("\n" + "="*60)
print("MÉTRIQUES D'ÉVALUATION")
print("="*60)
print(f"\nmAP@50: {metrics.box.map50:.4f}")
print(f"mAP@50-95: {metrics.box.map:.4f}")
print(f"Précision: {metrics.box.mp:.4f}")
print(f"Rappel: {metrics.box.mr:.4f}")
print(f"F1-Score: {2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr + 1e-6):.4f}")

# Afficher les métriques par classe
if hasattr(metrics.box, 'ap_class_index'):
    print(f"\n{'='*60}")
    print("MÉTRIQUES PAR CLASSE")
    print(f"{'='*60}")
    for i, class_idx in enumerate(metrics.box.ap_class_index):
        class_name = best_model.names[int(class_idx)]
        print(f"\nClasse: {class_name}")
        if hasattr(metrics.box, 'ap'):
            print(f"  AP@50: {metrics.box.ap50[i]:.4f}")
            print(f"  AP@50-95: {metrics.box.ap[i]:.4f}")

Évaluation du modèle sur l'ensemble de validation...
Ultralytics 8.3.229  Python-3.11.9 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti with Max-Q Design, 4096MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 118.927.3 MB/s, size: 13.2 KB)
val: Scanning D:\Cours\Master 2\Deep Learning\Exercice\Poubelles\dataset_detection_poubelle\valid\labels.cache... 32 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 32/32 32.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 2.6s/it 5.2s<12.8s
                   all         32         38      0.855      0.894      0.912      0.571
       poubelle_pleine         19         25      0.855       0.88      0.909      0.466
         poubelle_vide         13         13      0.855      0.909      0.915      0.676
Speed: 6.0ms preprocess, 22.6ms inference, 0.0ms loss, 15.7ms postprocess per imag

## 8. Matrice de confusion

In [38]:
# Générer des prédictions sur l'ensemble de validation pour la matrice de confusion
val_images_path = os.path.join(dataset_path, 'valid', 'images')
val_labels_path = os.path.join(dataset_path, 'valid', 'labels')

# Lire les annotations de validation
y_true = []
y_pred = []

print("Génération de la matrice de confusion...")

# Parcourir les images de validation
if os.path.exists(val_images_path):
    image_files = [f for f in os.listdir(val_images_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    for img_file in image_files:
        img_path = os.path.join(val_images_path, img_file)
        label_file = os.path.splitext(img_file)[0] + '.txt'
        label_path = os.path.join(val_labels_path, label_file)
        
        # Prédiction
        results = best_model.predict(img_path, verbose=False)
        
        # Lire les vraies étiquettes
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    true_class = int(line.strip().split()[0])
                    y_true.append(true_class)
                    
                    # Trouver la prédiction correspondante
                    if len(results[0].boxes) > 0:
                        pred_class = int(results[0].boxes.cls[0].item())
                        y_pred.append(pred_class)
                    else:
                        # Aucune détection - classe "background" ou -1
                        y_pred.append(-1)

# Créer la matrice de confusion
if len(y_true) > 0 and len(y_pred) > 0:
    # Filtrer les prédictions sans détection (-1)
    valid_indices = [i for i, pred in enumerate(y_pred) if pred != -1]
    y_true_filtered = [y_true[i] for i in valid_indices]
    y_pred_filtered = [y_pred[i] for i in valid_indices]
    
    if len(y_true_filtered) > 0:
        # Calculer la matrice de confusion
        cm = confusion_matrix(y_true_filtered, y_pred_filtered)
        
        # Obtenir les noms de classes
        class_names = [best_model.names[i] for i in sorted(set(y_true_filtered + y_pred_filtered))]
        
        # Afficher la matrice de confusion
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=class_names, 
                    yticklabels=class_names,
                    cbar_kws={'label': 'Nombre de prédictions'})
        plt.title('Matrice de Confusion - Détection de Poubelles', fontsize=14, fontweight='bold')
        plt.ylabel('Vraie Classe', fontsize=12)
        plt.xlabel('Classe Prédite', fontsize=12)
        plt.tight_layout()
        plt.show()
        
        # Rapport de classification
        print("\n" + "="*60)
        print("RAPPORT DE CLASSIFICATION")
        print("="*60)
        print(classification_report(y_true_filtered, y_pred_filtered, 
                                   target_names=class_names, 
                                   zero_division=0))
        
        # Calcul de l'accuracy
        accuracy = np.sum(np.array(y_true_filtered) == np.array(y_pred_filtered)) / len(y_true_filtered)
        print(f"\nAccuracy globale: {accuracy:.4f} ({accuracy*100:.2f}%)")
    else:
        print("Aucune prédiction valide trouvée pour générer la matrice de confusion.")
else:
    print("Impossible de générer la matrice de confusion - données insuffisantes.")

Génération de la matrice de confusion...


<Figure size 1000x800 with 2 Axes>


RAPPORT DE CLASSIFICATION
                 precision    recall  f1-score   support

poubelle_pleine       1.00      1.00      1.00        23
  poubelle_vide       1.00      1.00      1.00        13

       accuracy                           1.00        36
      macro avg       1.00      1.00      1.00        36
   weighted avg       1.00      1.00      1.00        36


Accuracy globale: 1.0000 (100.00%)


## 9. Analyse des résultats

In [ ]:
print("="*80)
print(" "*25 + "ANALYSE COMPLÈTE DES RÉSULTATS")
print("="*80)

# 1. Résumé de l'entraînement
print("\n📊 1. RÉSUMÉ DE L'ENTRAÎNEMENT")
print("-" * 80)
print(f"Temps d'entraînement: {training_hours}h {training_minutes}m {training_seconds}s")
print(f"Architecture: YOLOv9c")
print(f"Taille d'image: 640x640")
print(f"Batch size: 16")

# 2. Performance du modèle
print("\n📈 2. PERFORMANCE DU MODÈLE")
print("-" * 80)
if 'metrics' in locals():
    print(f"mAP@50: {metrics.box.map50:.4f} ({metrics.box.map50*100:.2f}%)")
    print(f"mAP@50-95: {metrics.box.map:.4f} ({metrics.box.map*100:.2f}%)")
    print(f"Précision: {metrics.box.mp:.4f} ({metrics.box.mp*100:.2f}%)")
    print(f"Rappel: {metrics.box.mr:.4f} ({metrics.box.mr*100:.2f}%)")
    f1 = 2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr + 1e-6)
    print(f"F1-Score: {f1:.4f} ({f1*100:.2f}%)")

# 3. Interprétation des résultats
print("\n💡 3. INTERPRÉTATION DES RÉSULTATS")
print("-" * 80)

if 'metrics' in locals():
    # Interprétation du mAP
    if metrics.box.map50 > 0.9:
        print("✅ Excellent mAP@50 (>90%) - Le modèle détecte très bien les poubelles")
    elif metrics.box.map50 > 0.7:
        print("✓ Bon mAP@50 (70-90%) - Performance satisfaisante")
    elif metrics.box.map50 > 0.5:
        print("⚠ mAP@50 moyen (50-70%) - Des améliorations sont possibles")
    else:
        print("❌ mAP@50 faible (<50%) - Le modèle nécessite des améliorations")
    
    # Interprétation Précision vs Rappel
    if metrics.box.mp > metrics.box.mr:
        print(f"\n📍 Précision > Rappel: Le modèle est conservateur")
        print("   → Peu de faux positifs, mais peut manquer certaines détections")
    elif metrics.box.mr > metrics.box.mp:
        print(f"\n📍 Rappel > Précision: Le modèle est permissif")
        print("   → Détecte beaucoup de poubelles, mais avec quelques faux positifs")
    else:
        print(f"\n📍 Précision ≈ Rappel: Modèle équilibré")

# 4. Recommandations
print("\n🎯 4. RECOMMANDATIONS")
print("-" * 80)

if 'metrics' in locals():
    if metrics.box.map50 < 0.7:
        print("• Augmenter le nombre d'époques d'entraînement")
        print("• Utiliser l'augmentation de données (data augmentation)")
        print("• Collecter plus d'images d'entraînement variées")
        print("• Essayer un modèle plus grand (yolov9e)")
    
    if metrics.box.mp < 0.7:
        print("• Ajuster le seuil de confiance pour réduire les faux positifs")
        print("• Vérifier la qualité des annotations")
    
    if metrics.box.mr < 0.7:
        print("• Réduire le seuil de confiance pour détecter plus d'objets")
        print("• Ajouter plus d'exemples de poubelles difficiles à détecter")



## 10. Visualisation des graphiques d'entraînement (générés par YOLO)

In [ ]:
# Afficher les graphiques générés automatiquement par YOLO
results_dir = 'runs/detect/poubelle_detection'

# Liste des graphiques à afficher
plot_files = [
    'results.png',
    'confusion_matrix.png',
    'PR_curve.png',
    'F1_curve.png'
]

fig, axes = plt.subplots(2, 2, figsize=(18, 14))
fig.suptitle('Graphiques d\'Entraînement YOLOv9', fontsize=16, fontweight='bold')

for idx, (ax, plot_file) in enumerate(zip(axes.flat, plot_files)):
    plot_path = os.path.join(results_dir, plot_file)
    
    if os.path.exists(plot_path):
        img = cv2.imread(plot_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax.imshow(img_rgb)
        ax.axis('off')
        ax.set_title(plot_file.replace('.png', '').replace('_', ' ').title(), 
                    fontsize=12, fontweight='bold')
    else:
        ax.text(0.5, 0.5, f'{plot_file}\nnon disponible', 
               ha='center', va='center', fontsize=12)
        ax.axis('off')

plt.tight_layout()
plt.show()

print(f"\n✅ Tous les graphiques sont disponibles dans: {results_dir}")

## 11. Prédiction sur de nouvelles images

In [ ]:
# Faire des prédictions sur des images de test
test_images_path = os.path.join(dataset_path, 'test', 'images')
results = best_model.predict(source=test_images_path, save=True, conf=0.5)

print(f"Prédictions sauvegardées dans: runs/detect/predict/")
print(f"Nombre d'images testées: {len(results)}")

## 12. Visualisation des prédictions

In [ ]:
# Fonction pour afficher quelques prédictions
def afficher_predictions(results, num_images=5):
    plt.figure(figsize=(15, 10))
    
    for i, result in enumerate(results[:num_images]):
        # Récupérer l'image avec les boîtes de détection
        img = result.plot()
        
        # Convertir BGR to RGB pour matplotlib
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        plt.subplot(2, 3, i+1)
        plt.imshow(img_rgb)
        plt.axis('off')
        
        # Afficher les classes détectées
        if len(result.boxes) > 0:
            classes = result.boxes.cls.cpu().numpy()
            names = [result.names[int(c)] for c in classes]
            plt.title(f"Détecté: {', '.join(set(names))}")
        else:
            plt.title("Aucune détection")
    
    plt.tight_layout()
    plt.show()

# Afficher les prédictions
afficher_predictions(results)

## 13. Fonction de prédiction sur une image unique

In [ ]:
def predire_poubelle(image_path, model):
    """
    Prédire si une poubelle est pleine ou vide
    
    Args:
        image_path: Chemin vers l'image à analyser
        model: Modèle YOLO entraîné
    
    Returns:
        dict: Résultats de la prédiction
    """
    results = model.predict(source=image_path, conf=0.5)
    
    predictions = []
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls)
            class_name = result.names[class_id]
            confidence = float(box.conf)
            
            predictions.append({
                'classe': class_name,
                'confiance': confidence,
                'bbox': box.xyxy.cpu().numpy()[0]
            })
    
    # Afficher l'image avec les détections
    img = results[0].plot()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title(f"Détections: {len(predictions)}")
    plt.show()
    
    return predictions

# Exemple d'utilisation
# image_test = "chemin/vers/votre/image.jpg"
# predictions = predire_poubelle(image_test, best_model)
# for pred in predictions:
#     print(f"Classe: {pred['classe']}, Confiance: {pred['confiance']:.2f}")

## 14. Sauvegarde et export du modèle

In [ ]:
# Le modèle est déjà sauvegardé dans runs/detect/poubelle_detection/weights/best.pt

# Export du modèle vers différents formats (optionnel)
# best_model.export(format='onnx')  # Pour ONNX
# best_model.export(format='tflite')  # Pour TensorFlow Lite
# best_model.export(format='torchscript')  # Pour TorchScript

print("Modèle entraîné disponible à: runs/detect/poubelle_detection/weights/best.pt")
print("\nPour utiliser le modèle plus tard:")
print("model = YOLO('runs/detect/poubelle_detection/weights/best.pt')")
print("results = model.predict('votre_image.jpg')")